In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## الخطوة 1: تعريف نماذج Pydantic للمخرجات المنظمة

تحدد هذه النماذج **المخطط** الذي ستعيده الوكلاء. استخدام `response_format` مع Pydantic يضمن:
- ✅ استخراج البيانات بشكل آمن من حيث النوع
- ✅ التحقق التلقائي
- ✅ عدم وجود أخطاء في التحليل من الردود النصية الحرة
- ✅ سهولة التوجيه الشرطي بناءً على الحقول


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## الخطوة 2: إنشاء أداة حجز الفنادق

هذه الأداة هي ما سيستخدمه **availability_agent** للتحقق من توفر الغرف. نستخدم الزخرفة `@ai_function` لـ:
- تحويل وظيفة Python إلى أداة قابلة للاستدعاء بواسطة الذكاء الاصطناعي
- إنشاء مخطط JSON تلقائيًا لـ LLM
- التعامل مع التحقق من صحة المعلمات
- تمكين الاستدعاء التلقائي بواسطة الوكلاء

لأغراض هذا العرض التوضيحي:
- **ستوكهولم، سياتل، طوكيو، لندن، أمستردام** → تتوفر غرف ✅
- **جميع المدن الأخرى** → لا توجد غرف ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## الخطوة 3: تعريف وظائف الشروط للتوجيه

تقوم هذه الوظائف بفحص استجابة الوكيل وتحديد المسار الذي يجب اتخاذه في سير العمل.

**النمط الرئيسي:**
1. التحقق مما إذا كانت الرسالة هي `AgentExecutorResponse`
2. تحليل المخرجات المهيكلة (نموذج Pydantic)
3. إرجاع `True` أو `False` للتحكم في التوجيه

سيقوم سير العمل بتقييم هذه الشروط على **الحواف** لتحديد أي منفذ يجب استدعاؤه بعد ذلك.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## الخطوة 4: إنشاء منفذ عرض مخصص

المنفذون هم مكونات سير العمل التي تقوم بإجراء التحويلات أو التأثيرات الجانبية. نستخدم الزخرفة `@executor` لإنشاء منفذ مخصص يعرض النتيجة النهائية.

**مفاهيم رئيسية:**
- `@executor(id="...")` - يسجل وظيفة كمنفذ لسير العمل
- `WorkflowContext[Never, str]` - تلميحات النوع للإدخال/الإخراج
- `ctx.yield_output(...)` - يُنتج النتيجة النهائية لسير العمل


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## الخطوة 5: تحميل متغيرات البيئة

قم بتكوين عميل LLM. يعمل هذا المثال مع:
- **نماذج GitHub** (الخطة المجانية باستخدام رمز GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## الخطوة 6: إنشاء وكلاء ذكاء اصطناعي بمخرجات منظمة

نقوم بإنشاء **ثلاثة وكلاء متخصصين**، كل واحد منهم مغلف بـ `AgentExecutor`:

1. **availability_agent** - يتحقق من توفر الغرف في الفندق باستخدام الأداة
2. **alternative_agent** - يقترح مدن بديلة (عندما لا تتوفر غرف)
3. **booking_agent** - يشجع على الحجز (عندما تكون الغرف متوفرة)

**الميزات الرئيسية:**
- `tools=[hotel_booking]` - يوفر الأداة للوكيل
- `response_format=PydanticModel` - يفرض إخراج JSON منظم
- `AgentExecutor(..., id="...")` - يغلف الوكيل لاستخدامه في سير العمل


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## الخطوة 7: بناء سير العمل باستخدام الحواف الشرطية

الآن نستخدم `WorkflowBuilder` لإنشاء الرسم البياني مع التوجيه الشرطي:

**هيكل سير العمل:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**الطرق الرئيسية:**
- `.set_start_executor(...)` - يحدد نقطة البداية
- `.add_edge(from, to, condition=...)` - يضيف حافة شرطية
- `.build()` - ينهي سير العمل


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## الخطوة 8: تشغيل حالة الاختبار 1 - مدينة بدون توفر (باريس)

لنقم باختبار مسار **عدم التوفر** من خلال طلب فنادق في باريس (التي لا تحتوي على غرف في محاكاتنا).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## الخطوة 9: تشغيل حالة الاختبار 2 - مدينة مع توفر (ستوكهولم)

الآن دعونا نختبر مسار **التوفر** من خلال طلب الفنادق في ستوكهولم (التي تحتوي على غرف في محاكاتنا).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## النقاط الرئيسية والخطوات التالية

### ✅ ما تعلمته:

1. **نمط WorkflowBuilder**
   - استخدم `.set_start_executor()` لتحديد نقطة البداية
   - استخدم `.add_edge(from, to, condition=...)` للتوجيه الشرطي
   - قم باستدعاء `.build()` لإنهاء إعداد سير العمل

2. **التوجيه الشرطي**
   - تقوم دوال الشرط بفحص `AgentExecutorResponse`
   - تحليل المخرجات المهيكلة لاتخاذ قرارات التوجيه
   - إرجاع `True` لتفعيل المسار، و`False` لتجاوزه

3. **دمج الأدوات**
   - استخدم `@ai_function` لتحويل دوال Python إلى أدوات ذكاء اصطناعي
   - الوكلاء يستدعون الأدوات تلقائيًا عند الحاجة
   - الأدوات تُرجع JSON يمكن للوكلاء تحليله

4. **المخرجات المهيكلة**
   - استخدم نماذج Pydantic لاستخراج البيانات بشكل آمن
   - قم بتعيين `response_format=MyModel` عند إنشاء الوكلاء
   - تحليل الردود باستخدام `Model.model_validate_json()`

5. **المنفذون المخصصون**
   - استخدم `@executor(id="...")` لإنشاء مكونات سير العمل
   - يمكن للمنفذين تحويل البيانات أو تنفيذ تأثيرات جانبية
   - استخدم `ctx.yield_output()` لإنتاج نتائج سير العمل

### 🚀 التطبيقات العملية:

- **حجز السفر**: التحقق من التوافر، اقتراح البدائل، مقارنة الخيارات
- **خدمة العملاء**: التوجيه بناءً على نوع المشكلة، المشاعر، الأولوية
- **التجارة الإلكترونية**: التحقق من المخزون، اقتراح البدائل، معالجة الطلبات
- **مراقبة المحتوى**: التوجيه بناءً على درجات السمية، إشارات المستخدم
- **سير العمل للموافقة**: التوجيه بناءً على المبلغ، دور المستخدم، مستوى المخاطرة
- **المعالجة متعددة المراحل**: التوجيه بناءً على جودة البيانات، اكتمالها

### 📚 الخطوات التالية:

- إضافة شروط أكثر تعقيدًا (معايير متعددة)
- تنفيذ الحلقات مع إدارة حالة سير العمل
- إضافة سير عمل فرعية للمكونات القابلة لإعادة الاستخدام
- التكامل مع واجهات برمجية حقيقية (حجز الفنادق، أنظمة المخزون)
- إضافة معالجة الأخطاء ومسارات احتياطية
- تصور سير العمل باستخدام أدوات التصور المدمجة



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
